# Sea surface temperature validation using gridded observations from NOAA

In [ ]:
variable = "temperature" 

In [ ]:
import pandas as pd
import importlib
import re
import warnings
import calendar
import copy
import oceanval
ff  = "../../matched/definitions.pkl"
import pickle
with open(ff, "rb") as f:
    definitions = pickle.load(f)

import glob
import geopandas as gpd
import jellyfish
import nctoolkit as nc
nc.options(parallel=True)
import os
import pickle
import numpy as np
import xarray as xr
from IPython.display import Markdown as md_markdown
import cmocean as cm
from tqdm import tqdm
import hvplot.pandas
from plotnine import *
from IPython.core.interactiveshell import InteractiveShell
from oceanval.tidiers import tidy_info
InteractiveShell.ast_node_interactivity = "all"

try:
    lon_lim = None
    lat_lim = None
except:
    lon_lim = None
    lat_lim = None
if lon_lim is None:
    lon_lim = [-180, 180]
if lat_lim is None:
    lat_lim = [-90, 90]

try:
    concise = False
except:
    concise = False

# in one line
from oceanval.tidiers import fix_basename, fix_unit, df_display, tidy_summary_paths, md, md_basic

warnings.filterwarnings('ignore')

%load_ext rpy2.ipython

test_status = False


i_figure = 1
i_table = 1
stamp = nc.session_info["stamp"]
out = ".trackers/" + stamp
fast_plot = False
if not os.path.exists(".trackers"):
    os.makedirs(".trackers")
# save out as empty file
with open(out, 'w') as f:
    f.write("")
nws = False

def bin_value(x, bin_res):
    return np.floor((x + bin_res / 2) / bin_res + 0.5) * bin_res - bin_res / 2

try:
    vv_name = variable
except:
    vv_name = "summary"
    variable = "summary"
Variable = variable.title()
if vv_name in ["benbio"]:
    compact = True
try:
    vv_name = definitions[variable].long_name 
except:
    pass

In [ ]:
stamp = nc.session_info["stamp"]
out = ".trackers/" + stamp + ".txt"
if not os.path.exists(".trackers"):
    os.makedirs(".trackers")
# save out as empty file
with open(out, 'w') as f:
    f.write("")

In [ ]:
source = "NOAA"
#domain = [x for x in glob.glob(f"../../matched/gridded/**/**/**_{variable.lower()}*.nc") if source in x][0].split("/")[-3]
domain = "nws"
sub_regions = "global"

In [ ]:

ff = glob.glob(f"../../matched/gridded/**/**_{variable.lower()}_*surface.nc")
ff_def = ff[0].replace(".nc", "_definitions.pkl")
with open(ff_def, "rb") as f:
    definitions = pickle.load(f)
model_variable = definitions[variable].model_variable    
md(f"**Matchup procedure**: The model and observations were matched up as follows. First, the model dataset was cropped by a small amount to make sure cells close to the boundary were removed. The model was then regridded to the observational grid if the observational grid was coarser using nearest neighbour. Only grid cells with model and observational data were maintained. The following model output was used to compare with the observational values: **{model_variable}**.")

In [ ]:
ff = [x for x in ff if f"{source}_" in x]
try:
    ff_summary = glob.glob(os.path.dirname(ff[0]) + "/*summary.pkl")[0]
    with open(ff_summary, 'rb') as f:
        vv_summary = pickle.load(f)
    clim_years = summary["clim"]
    clim_years = " (" + str(clim_years[0]) + "-" + str(clim_years[1]) + ") "
except:
    clim_years = ""
    pass

if len(ff) != 1:
    raise ValueError("Something is wrong with the file")
layer = os.path.basename(ff[0]).split("_")[-1].replace(".nc", "")
if layer == "surface":
    layer_long = "sea surface"
ff_vertical = ff[0].replace("_surface.nc", "_vertical.nc")
ds_model = nc.open_data(ff)
ds_model.subset(lon = lon_lim, lat = lat_lim)
# change the long_name
ds_model.set_precision("F32")
ds_model.subset(variable = "model")
ds_model.tmean("month")
ds_model.run()
new_name = definitions[variable].long_name 
new_name = new_name[0].upper() + new_name[1:]   
ds_model.set_longnames({ds_model.variables[0]: new_name})
ds_year = min(ds_model.years)
ds_model.set_year(ds_year)
ds_times = ds_model.times
df_times = pd.DataFrame({"year":[x.year for x in ds_times]}).groupby("year").size().reset_index()
df_times.columns = ["year", "count"]
years = list(df_times.query("count > 1").year)
ds_model.as_missing(0)
# if variable is doc, add 40
ds_model.run()
ds_annual = ds_model.copy()
ds_annual.tmean()
# ds_annual.set_longnames({ds_annual.variables[0]: Variable})

In [ ]:
md("## Baseline climatologies of sea surface temperature")

In [ ]:
# model climatology years can be derived from the vv_summary
try:
    clim_years = vv_summary["clim_years"]
    if clim_years[0] == clim_years[-1]:
        clim_text = f" The model climatology is calculated using the year **{clim_years[0]}**."
        clim_range = f" ({clim_years[0]}) "
    else:
        clim_text = f" The model climatology is calculated using the years **{clim_years[0]}-{clim_years[-1]}**."
        clim_range = f" ({clim_years[0]}-{clim_years[-1]}) "
except:
    clim_text = ""
    clim_range = ""
    pass
md_markdown(f"Climatologies of model and observational {layer_long} {vv_name} are shown in the figures below.{clim_text}")

In [ ]:
ff = glob.glob(f"../../matched/gridded/**/**_{variable}_*surface.nc")
ff = [x for x in ff if f"{source}_" in x]
ds_obs = nc.open_data(ff)
ds_obs.subset(variable = "observation")
ds_obs.subset(lon = lon_lim, lat = lat_lim)

ds_obs.run()
# changte the long_name
new_name = definitions[variable].long_name
new_name = new_name[0].upper() + new_name[1:]   
ds_obs.set_longnames({ds_obs.variables[0]: new_name})
ds_obs.set_precision("F32")
ds_obs.tmean("month")
ds_obs.run()

In [ ]:


if variable == "chlorophyll":
    transformation = "log10"
else:
    transformation = None 


ds_annual = ds_model.copy()
ds_annual.tmean()
# if "model" == ds_annual.variables[0]:
#     ds_annual.set_longnames({"model": Variable})
# else:
#     ds_annual.set_longnames({variable: Variable})

ds_annual.run()

#get the min/max lon lat with actual values in ds_annual
# save as [lon_min, lon_max] and [lat_min, lat_max]
coord_ranges = (
    ds_annual
    .to_dataframe()
    .reset_index()
    .dropna()
)
lon_name = [x for x in coord_ranges.columns if "lon" in x][0]
lat_name = [x for x in coord_ranges.columns if "lat" in x][0]
coord_ranges = (
    coord_ranges
    # rename the columns to lon and lat
    .rename(columns = {lon_name: "lon", lat_name: "lat"})
    #
    .loc[:,["lon", "lat"]]
    .agg(["min", "max"])
    .to_dict()
)

lon_min = coord_ranges["lon"]["min"]
lon_max = coord_ranges["lon"]["max"]
lat_min = coord_ranges["lat"]["min"]
lat_max = coord_ranges["lat"]["max"]
if lon_max < 90:
    ds_annual.subset(lon = [lon_min, lon_max], lat = [lat_min, lat_max])
fix_grid = False

In [ ]:
# Plot them on the same scale
ds_both = ds_model.copy()
ds_both.rename({ds_both.variables[0]: "model"})
ds_both.append(ds_obs)
ds_both.rename({ds_obs.variables[0]: "observation"})
ds_both.merge(match = "month")
ds_both.run()
ds_both.variables

#get the min/max lon lat with actual values in ds_annual
# save as [lon_min, lon_max] and [lat_min, lat_max]
coord_ranges = (
    ds_both
    .to_dataframe()
    .reset_index()
    .dropna()
)
lon_name = [x for x in coord_ranges.columns if "lon" in x][0]
lat_name = [x for x in coord_ranges.columns if "lat" in x][0]
coord_ranges = (
    coord_ranges
    # rename the columns to lon and lat
    .rename(columns = {lon_name: "lon", lat_name: "lat"})
    #
    .loc[:,["lon", "lat"]]
    .agg(["min", "max"])
    .to_dict()
)

lat_min = coord_ranges["lat"]["min"]
lat_max = coord_ranges["lat"]["max"]
if lon_max < 90:
    ds_both.subset(lon = [lon_min, lon_max], lat = [lat_min, lat_max])

if fix_grid:
    ds_both.to_latlon(lon = [lon_min , lon_max], lat = [lat_min, lat_max], res = [lon_res, lat_res])

In [ ]:
import matplotlib.pyplot as plt

plt.subplots_adjust(wspace=20, hspace=20)

fig = plt.figure(figsize=(14, 14))

# Create 1x2 Grid

gs = fig.add_gridspec(nrows=1, ncols=2, wspace = 0.35, hspace = 0)

# get limits..

ds_both.tmean()

z_min = (
    ds_both
    .to_dataframe()
    .reset_index()
    .dropna()
    .loc[:,["model", "observation"]]
    # get the 2nd percentils
    # .groupby("variable")
    .quantile(0.02)
    .min()
)

# get the 98th percentils

z_max = (
    ds_both
    .to_dataframe()
    .reset_index()
    .dropna()
    .loc[:,["model", "observation"]]
    .quantile(0.98)
    .max()
)
# fix the units

ds_both.set_units({"observation": ds_both.contents.query("variable == 'model'").reset_index().unit.values[0]})
# chang longname
# try:
#     ds_both.set_longnames({"observation": ds_both.contents.query("variable == 'model'").reset_index().long_name.values[0]}) 
# except:
#     ds_both.set_longnames({"observation": ds_both.contents.query("variable == 'observation'").reset_index().long_name.values[0]})  
# ditch #Modelled and Observed from long names
the_contents = ds_both.contents
for vv in the_contents.variable:
    long_name = the_contents.query(f"variable == '{vv}'").reset_index().long_name.values[0]
    if "Modelled" in long_name:
        long_name = long_name.replace("Modelled", "").strip().replace("surface", "Surface")
    if "Observed" in long_name:
        long_name = long_name.replace("Observed", "").strip().replace("surface", "Surface")
    ds_both.set_longnames({vv: new_name})

ds_both.pub_plot(variable  = "model", limits = [z_min, z_max], title = "Model", fig = fig, gs = gs[0,0], trans = transformation)



ds_both.pub_plot(variable  = "observation", limits = [z_min, z_max], title = "Observation", fig = fig, gs = gs[0,1], trans = transformation)

In [ ]:
fig




In [ ]:
md(f"**Figure {i_figure}**: Annual average {layer} {vv_name} from the model {clim_range} and observations. Data is limited to the 2nd and 98th percentile of the combined model and observational data. Arrows indicate that values can exceed the colorbar limits.") 
i_figure += 1

In [ ]:
md(f"## Assessing model bias for {layer} {vv_name}")
#
# A critical metric for model performance is the bias between model and observed values. Here the bias is calculated as the mean difference between model and observed values. A positive bias indicates that the model overestimates the observed values, while a negative bias indicates that the model underestimates the observed values. 



In [ ]:
ds_bias = ds_model.copy()
ds_bias - ds_obs
ds_bias.tmean()
ds_bias.run()
# model plot

df_bias = ds_bias.to_dataframe().reset_index()
if "model" == ds_bias.variables[0]:
    ds_bias.set_longnames({"model": Variable + " bias"})
else:
    ds_bias.set_longnames({variable: Variable + " bias"})

# get th
ds_ave = ds_bias.copy()
ds_ave.spatial_mean()
ds_ave.rename({ds_ave.variables[0]: "bias"})
ave_bias = ds_ave.to_dataframe().bias.values[0]
ds_summary = ds_bias.copy()
ds_summary > 0 
ds_summary.spatial_mean()
ds_summary.rename({ds_summary.variables[0]: "bias"})
positive_bias = ds_summary.to_dataframe().bias.values[0] * 100
# as a percentage to 1 dp
positive_bias = round(positive_bias, 1)
the_unit = ds_summary.contents.unit.values[0]

md(f"Figure {i_figure} shows the average bias of {layer} {vv_name} simulated by the model. A positive bias indicates that the model overestimates the observation, while a negative bias indicates that the model overpredicts the observation.") 
if positive_bias > 50:
    positive_bias = str(positive_bias) + "%"
    md(f"The spatial average bias of {layer} {vv_name} is {ave_bias:.2f} {the_unit}. Overall, the model overestimates the observations in {positive_bias} of the model domain.")
else:
    negative_bias = str(100-positive_bias) + "%"
    md(f"The spatial average bias of {layer} {vv_name} is {ave_bias:.2f} {the_unit}. Overall, the model underestimates the observations in {negative_bias} of the model domain.")



In [ ]:

if fix_grid:
    ds_bias_1 = ds_bias.copy()
    ds_bias_1.to_latlon(lon = [lon_min , lon_max], lat = [lat_min, lat_max], res = [lon_res, lat_res])
    # change the name
    ds_bias_1.set_longnames({ds_bias_1.variables[0]: "Model bias"})
    ds_bias_1.pub_plot(robust = True)
else:
    # change the name
    ds_bias.set_longnames({ds_bias.variables[0]: "Model bias"})
    ds_bias.pub_plot(robust = True)

In [ ]:
md(f"**Figure {i_figure}**: Bias of {layer} {vv_name} from the model. A positive bias indicates that the model overestimates the observation.  For clarity, the colorbar is limited to the 2nd and 98th percentile of the data.")
i_figure += 1

In [ ]:
md(f"## Can the model reproduce seasonality of {layer_long} {vv_name}?")

md(f"The ability of the model to reproduce seasonality of {layer_long} {vv_name} was assessed by comparing the modelled and observed seasonal cycle of {vv_name}. First, we derive a monthly climatology for the model data. Then, we calculate the Pearson correlation coefficient between the modelled and observed {vv_name} at each grid cell.")


md("Note: we are only assessing the ability of the model to reproduce the ability of the model to reproduce seasonal changes, not long-term trends.")

In [ ]:
ds1 = ds_model.copy()
ds1.cdo_command("setname,model")
ds1.run()
ds2 = ds_obs.copy()
ds2.cdo_command("setname,observation")
ds2.run()
ds_cor = nc.open_data([ds1.current[0], ds2.current[0]])
ds_cor.merge(match=["month"])
ds_cor.run()
ds_ts = ds_cor.copy()
ds_cor.cor_time("model", "observation")
title = f"Seasonal temporal correlation between {variable} for model and observations"
ds_cor.run()

# output to nc

out = f"../../results/temporals/{variable}_cor_{source}.nc"
if not os.path.exists(os.path.dirname(out)):
    os.makedirs(os.path.dirname(out))
ds_cor.to_nc(out, zip = True, overwrite = True)

# output to csv

if False:
    df_cor = ds_cor.to_dataframe().reset_index()
    df_cor = df_cor.dropna()
    out = f"../../results/temporals/{variable}_cor_{source}.csv"
    if not os.path.exists(os.path.dirname(out)):
        os.makedirs(os.path.dirname(out))
    df_cor.to_csv(out, index = False)

In [ ]:

df_cor = ds_cor.to_dataframe().reset_index()
if fix_grid:
    ds_cor_plot = ds_cor.copy()
    ds_cor_plot.to_latlon(lon = [lon_min , lon_max], lat = [lat_min, lat_max], res = [lon_res, lat_res])
    ds_cor_plot.pub_plot("cor")
else:
    ds_cor.pub_plot("cor")



In [ ]:
md(f"**Figure {i_figure}**: Seasonal temporal correlation between model and observations for {layer} {vv_name}. This is the Pearson correlation coefficient between climatological monthly mean values in the model and observations.")
i_figure += 1

In [ ]:
lon_range = lon_max - lon_min
lat_range = lat_max - lat_min

global_grid = False
if lon_range > 340:
    if lat_range > 160:
        global_grid = True


data_path = str(importlib.resources.files("oceanval").joinpath(f"data/{sub_regions}_subdomains.nc"))

regional = False

if sub_regions in ["nwes", "global"]: 
    ds_regions = nc.open_data(data_path, checks = False)
    if os.path.exists(data_path):
        regional = True
    # pull this in from the package data
    
    ds_regions.as_missing(0)
    ds_regions.set_fill(-9999)
    ds_regions.run()
    ds_regions.regrid(ds_model, method = "nn")
    regions_contents = ds_regions.contents
    
    # figure out if you can sensibly do a regional analysis for nws
    grid = pd.read_csv("../../matched/model_grid.csv")
    lon = grid.loc[:,[x for x in grid.columns if "lon" in x]].values
    lon = np.unique(lon)
    lon.sort()
    lat = grid.loc[:,[x for x in grid.columns if "lat" in x]].values
    lat = np.unique(lat)
    lat.sort()
    # get unique values in grid and sort them
    lon = np.unique(lon)
    lon.sort()
    lon_min = lon.min()
    lon_max = lon.max()
    lat_min = lat.min()
    lat_max = lat.max()

In [ ]:
#shape = gpd.read_file(f"{data_dir}/mapping/TM_WORLD_BORDERS-0.3.shp")
mod_var = ds_model.variables[0]
obs_var = ds_obs.variables[0]
# create xlim using mod_var
try:
    time_name = [x for x in list(ds_model.to_xarray().coords) if "time" in x][0]
except:
    time_name = None
    concise = True
if len(ds_model.times) == 1:
    concise = True
    time_name = None
    model_unit = ""
    raw_extent = "foo"
    xlim = "bar"
    ylim = "ba"
if time_name is not None:
    lon_name = [x for x in list(ds_obs.to_xarray().coords) if "lon" in x][0]
    lat_name = [x for x in list(ds_obs.to_xarray().coords) if "lat" in x][0]

    df_model = (
        ds_model
        .to_dataframe()
        .reset_index()
        .rename(columns = {time_name: "time"})
        .rename(columns = {lon_name: "lon"})
        .rename(columns = {lat_name: "lat"})
        .assign(month = lambda x: x.time.dt.month)
        .loc[:,["lon", "lat", "month", mod_var ]]
        .rename(columns = {mod_var: "model"})
        .dropna()
    )

    xlim = np.array([df_model.lon.min(), df_model.lon.max()])
    ylim = np.array([df_model.lat.min(), df_model.lat.max()])

    df_obs = (
        ds_obs
        .to_dataframe()
        .reset_index()
    )

    time_name = [x for x in list(ds_obs.to_xarray().coords) if "time" in x][0]
    lon_name = [x for x in list(ds_obs.to_xarray().coords) if "lon" in x][0]
    lat_name = [x for x in list(ds_obs.to_xarray().coords) if "lat" in x][0]


    df_obs = (
        df_obs
        .rename(columns = {time_name: "time"}  )
        .rename(columns = {lon_name: "lon"}  )
        .rename(columns = {lat_name: "lat"}  )
        .assign(month = lambda x: x.time.dt.month)
        .loc[:,["lon", "lat", "month", obs_var ]]
        .rename(columns = {obs_var: "observation"})
        .dropna()

    )

    df_diff = (
        df_model
        .merge(df_obs, on = ["lon", "lat", "month"])
        .assign(diff = lambda x: x.model - x.observation)
    )
    try:
        model_unit = ds_model.contents.unit[0]
        model_unit = fix_unit(model_unit)
    except:
        model_unit = ds_obs.contents.unit[0]
        model_unit = fix_unit(model_unit)
    from oceanval.utils import get_extent
    raw_extent = get_extent(ds_annual[0])
    if np.abs(raw_extent[0] - df_model.lon.min()) > 3:
        # convert longitude to -180-180
        df_model["lon" ] = [x if x < 180 else x -360 for x in df_model.lon]
        df_obs["lon" ] = [x if x < 180 else x -360 for x in df_obs.lon]
        df_diff["lon" ] = [x if x < 180 else x -360 for x in df_diff.lon]

    # generate a temporary csv file name in /tmp
    # create adhoc dir if not
    if not os.path.exists("adhoc/tmp"):
        os.makedirs("adhoc/tmp")
    tmp_csv = f"adhoc/df_obs_model.csv"
    df_obs.to_csv(tmp_csv)
    tmp_csv = f"adhoc/df_model_model.csv"
    df_model.to_csv(tmp_csv)
    tmp_csv = f"adhoc/df_diff_model.csv"
    df_diff.to_csv(tmp_csv)

In [ ]:

    if not concise:
        md(f"The seasonal cycles of simulated and observed {vv_name} are compared in Figure {i_figure} below. This figure shows the model and observation average in each month of the year, and the differences between the two each month") 



In [ ]:
%%capture --no-display
%%R -i model_unit  -w 800 -h 600 -i variable -i raw_extent -i concise -r 100
options(warn=-1)


if(concise == FALSE){

fixed_scale = "False"

if(fixed_scale == "False"){
    fixed_scale = FALSE
}
if(fixed_scale == "True"){
    fixed_scale = TRUE
}


library(tidyverse, warn.conflicts = FALSE)
library(cowplot, warn.conflicts = FALSE)
#library(tidyr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)

df_model <- read_csv("adhoc/df_model_model.csv")


min_month = min(df_model$month)
if("month" %in% colnames(df_model) & min_month < 7){

    model_unit = str_replace(model_unit, "/m\\^3", "m<sup>-3</sup>")
    model_unit <- str_replace(model_unit, "/kg", "kg<sup>-1</sup>")

    df_model_raw <- drop_na(df_model)
    df_obs <- read_csv("adhoc/df_obs_model.csv")
    df_diff <- read_csv("adhoc/df_diff_model.csv")


#df_model_raw <- drop_na(df_model)
df_obs_raw <- drop_na(df_obs)
df_diff_raw <- drop_na(df_diff)

if (variable == "temperature" )
    model_unit = "°C"

df_model <- drop_na(df_model)
df_model <- df_model %>%
    filter(month %in% c(1,2,3,4,5,6))
df_obs <- df_obs %>%
    filter(month %in% c(1,2,3,4,5,6))
df_diff <- df_diff %>%
    filter(month %in% c(1,2,3,4,5,6))
# 

df_model_raw %>%
    group_by(month) %>%
    summarize(model_98 = quantile(model, probs = 0.98)) %>%
    ungroup() %>%
    summarize(model_98 = mean(model_98)) %>%
    ungroup() %>%
    pull(model_98) -> model_98

obs_98 <- df_obs_raw %>%
    group_by(month) %>%
    summarize(obs_98 = quantile(observation, probs = 0.98)) %>%
    ungroup() %>%
    summarize(obs_98 = mean(obs_98)) %>%
    ungroup() %>%
    pull(obs_98)

if(fixed_scale){
    model_98 = max(c(model_98, obs_98))
    obs_98 = max(c(model_98, obs_98))
    # calculate the 2nd percentile of model and obs
    model_02 <- df_model_raw %>%
        group_by(month) %>%
        summarize(model_02 = quantile(model, probs = 0.02)) %>%
        ungroup() %>%
        summarize(model_02 = mean(model_02)) %>%
        ungroup() %>%
        pull(model_02)
    obs_02 <- df_obs_raw %>%
        group_by(month) %>%
        summarize(obs_02 = quantile(observation, probs = 0.02)) %>%
        ungroup() %>%
        summarize(obs_02 = mean(obs_02)) %>%
        ungroup() %>%
        pull(obs_02)
    model_02 = min(c(model_02, obs_02))
    obs_02 = min(c(model_02, obs_02))

}

df_model <- df_model %>%
    mutate(model = ifelse(model > model_98, model_98, model))

df_obs <- df_obs %>%
    mutate(observation = ifelse(observation > obs_98, obs_98, observation))
if (fixed_scale){
    df_model <- df_model %>%
        mutate(model = ifelse(model < model_02, model_02, model))

    df_obs <- df_obs %>%
        mutate(observation = ifelse(observation < obs_02, obs_02, observation))
}

diff_02 <- df_diff_raw %>%
    group_by(month) %>%
    summarize(diff_02 = quantile(diff, probs = 0.02)) %>% 
    ungroup() %>%
    summarize(diff_02 = mean(diff_02)) %>%
    ungroup() %>%
    pull(diff_02)

diff_98 <- df_diff_raw %>%
    group_by(month) %>%
    summarize(diff_98 = quantile(diff, probs = 0.98)) %>%
    ungroup() %>%
    summarize(diff_98 = mean(diff_98)) %>%
    ungroup() %>%
    pull(diff_98)

df_diff <- df_diff %>% 
    mutate(diff = ifelse(diff < diff_02, diff_02, diff)) %>%
    mutate(diff = ifelse(diff > diff_98, diff_98, diff))



xlim = c(min(df_model$lon), max(df_model$lon))
ylim = c(min(df_model$lat), max(df_model$lat))

world_map <- map_data("world") 

# get 98th percentile of df_model$

# function to convert month number in int to month name
month_name <- function(x){
    month.abb[x]
}

# vectorize month_name function
# 
month_name <- Vectorize(month_name) 



# convert month number to month name in dataframes

# 

df_model <- df_model %>%
    mutate(month = month_name(month))

df_obs <- df_obs %>%
    mutate(month = month_name(month))

df_diff <- df_diff %>%
    mutate(month = month_name(month))

# first six months of the year in dataframes

df_model <- df_model %>%
    filter(month %in% c("Jan", "Feb", "Mar", "Apr", "May", "Jun"))

df_obs <- df_obs %>%
    filter(month %in% c("Jan", "Feb", "Mar", "Apr", "May", "Jun"))

df_diff <- df_diff %>%  
    filter(month %in% c("Jan", "Feb", "Mar", "Apr", "May", "Jun"))


# change month to suitable factor in dataframes

df_model$month <- factor(df_model$month, levels = c("Jan", "Feb", "Mar", "Apr", "May", "Jun"))

df_obs$month <- factor(df_obs$month, levels = c("Jan", "Feb", "Mar", "Apr", "May", "Jun"))

df_diff$month <- factor(df_diff$month, levels = c("Jan", "Feb", "Mar", "Apr", "May", "Jun"))


# Create sensible lon/lat labels based on the min/min lon and lat
# This needs to work on any data, including global data
lon_breaks = c(xlim[1], xlim[1] + 10, 0, xlim[2] - 10)
lat_breaks = c(ylim[1], ylim[1] + 5, ylim[1] + 10, ylim[1] + 15, ylim[2])
lon_labels = c(paste0(round(xlim[1]), "°W"), paste0(round(xlim[1] + 10), "°W"), "0°", paste0(round(xlim[2] - 10), "°E"))
lat_labels = c(paste0(round(ylim[1]), "°N"), paste0(round(ylim[1] + 5), "°N"), paste0(round(ylim[1] + 10), "°N"), paste0(round(ylim[1] + 15), "°N"), paste0(round(ylim[2]), "°N"))


gg1 = ggplot(df_model)+
    geom_tile(aes(x = lon, y = lat, fill = model))+
    facet_wrap(~month, ncol = 6)+
    coord_cartesian(xlim = xlim, ylim = ylim, expand = FALSE)+ 
    theme_bw(base_size = 12)+
    theme(legend.title = ggtext::element_markdown(angle = -90), legend.title.align = 0.5)+
    scale_fill_viridis_c(guide  = guide_colourbar(title.position = "right"))+
    labs(x = NULL, y = NULL, title = "Model", fill = model_unit)


gg1 <- gg1 +
    # remove the x and y axis totally
    theme(axis.text.x = element_blank(), axis.text.y = element_blank(),
          axis.ticks.x = element_blank(), axis.ticks.y = element_blank(),
          axis.title.x = element_blank(), axis.title.y = element_blank()) +
    labs(x = "", y = "") 

gg1 <- gg1 + 
    geom_polygon(data = world_map, aes(x = long, y = lat, group = group), fill = "grey", colour = "grey")

gg2 = ggplot(df_obs)+
    geom_tile(aes(x = lon, y = lat, fill = observation))+
    facet_wrap(~month, ncol = 6)+
    coord_cartesian(xlim = xlim, ylim = ylim, expand = FALSE)+ 
    scale_fill_viridis_c(guide  = guide_colourbar(title.position = "right"))+
    theme_bw(base_size = 12)+
    theme(legend.title = ggtext::element_markdown(angle = -90), legend.title.align = 0.5)+
    labs(x = NULL, y = NULL, title = "Observation", fill = model_unit)


gg2 <- gg2 +
    # remove the x and y axis totally
    theme(axis.text.x = element_blank(), axis.text.y = element_blank(),
          axis.ticks.x = element_blank(), axis.ticks.y = element_blank(),
          axis.title.x = element_blank(), axis.title.y = element_blank()) +
    labs(x = "", y = "") 

gg2 <-  gg2 + 
    geom_polygon(data = world_map, aes(x = long, y = lat, group = group), fill = "grey", colour = "grey")

gg3 = ggplot(df_diff)+
geom_tile(aes(x = lon, y = lat, fill = diff))+
facet_wrap(~month, ncol = 6)+
coord_cartesian(xlim = xlim, ylim = ylim, expand = FALSE)+ 
theme_bw(base_size = 12)+
scale_fill_gradient2(guide  = guide_colourbar(title.position = "right"), low = "blue", high = "red")+
theme(legend.title = ggtext::element_markdown(angle = -90), legend.title.align = 0.5)+
labs(x = NULL, y = NULL, title = "Model - Observation", fill = model_unit)


gg3 <- gg3 +
    # remove the x and y axis totally
    theme(axis.text.x = element_blank(), axis.text.y = element_blank(),
          axis.ticks.x = element_blank(), axis.ticks.y = element_blank(),
          axis.title.x = element_blank(), axis.title.y = element_blank()) +
    labs(x = "", y = "") 


gg3 <- gg3 + 
        geom_polygon(data = world_map, aes(x = long, y = lat, group = group), fill = "grey", colour = "grey")


cowplot::plot_grid(gg1, gg2, gg3, ncol = 1)

}
}



In [ ]:
%%capture --no-display
%%R -i model_unit -w 800 -h 600 -i variable -i raw_extent -i concise -r 100
options(warn=-1)

if (concise == FALSE){

    fixed_scale = "False"   
    if (fixed_scale == "False")
        fixed_scale = FALSE
    if (fixed_scale == "True")
        fixed_scale = TRUE
    
library(tidyverse, warn.conflicts = FALSE)
library(cowplot, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)

df_model <- read_csv("adhoc/df_model_model.csv")

max_month = max(df_model$month)
if("month" %in% colnames(df_model) & max_month > 6){

    df_model_raw <- drop_na(df_model)
    df_obs <- read_csv("adhoc/df_obs_model.csv")
    df_diff <- read_csv("adhoc/df_diff_model.csv")


df_obs_raw <- drop_na(df_obs)
df_diff_raw <- drop_na(df_diff)

if (variable == "temperature" )
    model_unit = "°C"

    model_unit = str_replace(model_unit, "/m\\^3", "m<sup>-3</sup>")
    # fix /kg to kg^-2
    model_unit <- str_replace(model_unit, "/kg", "kg<sup>-1</sup>")

df_model <- drop_na(df_model)
df_model <- df_model %>%
    filter(month > 6) 
df_obs <- df_obs %>%
    filter(month > 6) 
df_diff <- df_diff %>%
    filter(month > 6) 

df_model_raw %>%
    group_by(month) %>%
    summarize(model_98 = quantile(model, probs = 0.98)) %>%
    ungroup() %>%
    summarize(model_98 = mean(model_98)) %>%
    ungroup() %>%
    pull(model_98) -> model_98

obs_98 <- df_obs_raw %>%
    group_by(month) %>%
    summarize(obs_98 = quantile(observation, probs = 0.98)) %>%
    ungroup() %>%
    summarize(obs_98 = mean(obs_98)) %>%
    ungroup() %>%
    pull(obs_98)

if (fixed_scale){
    model_98 = max(c(model_98, obs_98))
    obs_98 = max(c(model_98, obs_98))
    # calculate the 2nd percentile of model and obs
    model_02 <- df_model_raw %>%
        group_by(month) %>%
        summarize(model_02 = quantile(model, probs = 0.02)) %>%
        ungroup() %>%
        summarize(model_02 = mean(model_02)) %>%
        ungroup() %>%
        pull(model_02)
    obs_02 <- df_obs_raw %>%
        group_by(month) %>%
        summarize(obs_02 = quantile(observation, probs = 0.02)) %>%
        ungroup() %>%
        summarize(obs_02 = mean(obs_02)) %>%
        ungroup() %>%
        pull(obs_02)

}


df_model <- df_model %>%
    mutate(model = ifelse(model > model_98, model_98, model))

    if (fixed_scale){
        df_model <- df_model %>%
            mutate(model = ifelse(model < model_02, model_02, model))
    }



df_obs <- df_obs %>%
    mutate(observation = ifelse(observation > obs_98, obs_98, observation))

if (fixed_scale){
    df_obs <- df_obs %>%
        mutate(observation = ifelse(observation < obs_02, obs_02, observation))
    }

diff_02 <- df_diff_raw %>%
    group_by(month) %>%
    summarize(diff_02 = quantile(diff, probs = 0.02)) %>% 
    ungroup() %>%
    summarize(diff_02 = mean(diff_02)) %>%
    ungroup() %>%
    pull(diff_02)

diff_98 <- df_diff_raw %>%
    group_by(month) %>%
    summarize(diff_98 = quantile(diff, probs = 0.98)) %>%
    ungroup() %>%
    summarize(diff_98 = mean(diff_98)) %>%
    ungroup() %>%
    pull(diff_98)

df_diff <- df_diff %>% 
    mutate(diff = ifelse(diff < diff_02, diff_02, diff)) %>%
    mutate(diff = ifelse(diff > diff_98, diff_98, diff))



xlim = c(min(df_model$lon), max(df_model$lon))
ylim = c(min(df_model$lat), max(df_model$lat))

world_map <- map_data("world") 

# get 98th percentile of df_model$

# function to convert month number in int to month name
month_name <- function(x){
    month.abb[x]
}

# vectorize month_name function
# 
month_name <- Vectorize(month_name) 



# convert month number to month name in dataframes

# 

df_model <- df_model %>%
    mutate(month = month_name(month))

df_obs <- df_obs %>%
    mutate(month = month_name(month))

df_diff <- df_diff %>%
    mutate(month = month_name(month))

# first six months of the year in dataframes


df_model <- df_model %>%    
    filter(month %in% c("Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))

df_obs <- df_obs %>%
    filter(month %in% c("Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))

df_diff <- df_diff %>%
    filter(month %in% c("Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))

# change month to suitable factor in dataframes

df_model$month <- factor(df_model$month, levels = c("Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))

df_obs$month <- factor(df_obs$month, levels = c("Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))

df_diff$month <- factor(df_diff$month, levels = c("Jul", "Aug", "Sep", "Oct", "Nov", "Dec"))

# Create sensible lon/lat labels based on the min/min lon and lat
# This needs to work on any data, including global data
lon_breaks = c(xlim[1], xlim[1] + 10, 0, xlim[2] - 10)
lat_breaks = c(ylim[1], ylim[1] + 5, ylim[1] + 10, ylim[1] + 15, ylim[2])
lon_labels = c(paste0(round(xlim[1]), "°W"), paste0(round(xlim[1] + 10), "°W"), "0°", paste0(round(xlim[2] - 10), "°E"))
lat_labels = c(paste0(round(ylim[1]), "°N"), paste0(round(ylim[1] + 5), "°N"), paste0(round(ylim[1] + 10), "°N"), paste0(round(ylim[1] + 15), "°N"), paste0(round(ylim[2]), "°N"))



gg1 = ggplot(df_model)+
geom_tile(aes(x = lon, y = lat, fill = model))+
facet_wrap(~month, ncol = 6)+
coord_cartesian(xlim = xlim, ylim = ylim, expand = FALSE)+ 
theme_bw(base_size = 12)+
theme(legend.title = ggtext::element_markdown(angle = -90), legend.title.align = 0.5)+
scale_fill_viridis_c(guide  = guide_colourbar(title.position = "right"))+
labs(x = NULL, y = NULL, title = "Model", fill = model_unit)


gg1 <- gg1 + 
        geom_polygon(data = world_map, aes(x = long, y = lat, group = group), fill = "grey", colour = "grey")

gg1 <- gg1 +
    # remove the x and y axis totally
    theme(axis.text.x = element_blank(), axis.text.y = element_blank(),
          axis.ticks.x = element_blank(), axis.ticks.y = element_blank(),
          axis.title.x = element_blank(), axis.title.y = element_blank()) +
    labs(x = "", y = "") 


gg2 = ggplot(df_obs)+
geom_tile(aes(x = lon, y = lat, fill = observation))+
facet_wrap(~month, ncol = 6)+
coord_cartesian(xlim = xlim, ylim = ylim, expand = FALSE)+ 
scale_fill_viridis_c(guide  = guide_colourbar(title.position = "right"))+
theme_bw(base_size = 12)+
theme(legend.title = ggtext::element_markdown(angle = -90), legend.title.align = 0.5)+
labs(x = NULL, y = NULL, title = "Observation", fill = model_unit)

gg2 <- gg2 + 
        geom_polygon(data = world_map, aes(x = long, y = lat, group = group), fill = "grey", colour = "grey")

gg2 <- gg2 +
    # remove the x and y axis totally
    theme(axis.text.x = element_blank(), axis.text.y = element_blank(),
          axis.ticks.x = element_blank(), axis.ticks.y = element_blank(),
          axis.title.x = element_blank(), axis.title.y = element_blank()) +
    labs(x = "", y = "") 

gg3 = ggplot(df_diff)+
geom_tile(aes(x = lon, y = lat, fill = diff))+
facet_wrap(~month, ncol = 6)+
coord_cartesian(xlim = xlim, ylim = ylim, expand = FALSE)+ 
theme_bw(base_size = 12)+
scale_fill_gradient2(guide  = guide_colourbar(title.position = "right"), low = "blue", high = "red")+
theme(legend.title = ggtext::element_markdown(angle = -90), legend.title.align = 0.5)+
labs(x = NULL, y = NULL, title = "Model - Observation", fill = model_unit)


gg3 <- gg3 + 
        geom_polygon(data = world_map, aes(x = long, y = lat, group = group), fill = "grey", colour = "grey")

gg3 <- gg3 +
    # remove the x and y axis totally
    theme(axis.text.x = element_blank(), axis.text.y = element_blank(),
          axis.ticks.x = element_blank(), axis.ticks.y = element_blank(),
          axis.title.x = element_blank(), axis.title.y = element_blank()) +
    labs(x = "", y = "") 

gg1 <- gg1 + 
   theme(plot.margin = margin(t = 0,  # Top margin
                             r = 0,  # Right margin
                             b = 0,  # Bottom margin
                             l = 0)) # Left ggmargin 

# figure out if it's a global file
if(abs(raw_extent[1] - raw_extent[2]) > 350){
    gg1 <- gg1 + 
    scale_x_continuous(breaks = c(-180, -90, 0, 90, 180), labels = c("180°W", "90°W", "0°", "90°E", "180°E"))+
    scale_y_continuous(breaks = c(-90, -45, 0, 45, 90), labels = c("90°S", "45°S", "0°", "45°N", "90°N"))

    gg2 <- gg2 +
    scale_x_continuous(breaks = c(-180, -90, 0, 90, 180), labels = c("180°W", "90°W", "0°", "90°E", "180°E"))+
    scale_y_continuous(breaks = c(-90, -45, 0, 45, 90), labels = c("90°S", "45°S", "0°", "45°N", "90°N"))

    gg3 <- gg3 +
    scale_x_continuous(breaks = c(-180, -90, 0, 90, 180), labels = c("180°W", "90°W", "0°", "90°E", "180°E"))+
    scale_y_continuous(breaks = c(-90, -45, 0, 45, 90), labels = c("90°S", "45°S", "0°", "45°N", "90°N"))

}

# appropriate plotting for nws 
if((raw_extent[1] > -30) & (raw_extent[2] < 20)){
    gg1 <- gg1 + 
    scale_x_continuous(breaks = c(-20, -10, 0, 10), labels = c("20°W", "10°W", "0°", "10°E"))+
    scale_y_continuous(breaks = c(45, 50, 55, 60, 65), labels = c("45°N", "50°N", "55°N", "60°N", "65°N"))
    gg2 <- gg2 + 
    scale_x_continuous(breaks = c(-20, -10, 0, 10), labels = c("20°W", "10°W", "0°", "10°E"))+
    scale_y_continuous(breaks = c(45, 50, 55, 60, 65), labels = c("45°N", "50°N", "55°N", "60°N", "65°N"))
    gg3 <- gg3 +
    scale_x_continuous(breaks = c(-20, -10, 0, 10), labels = c("20°W", "10°W", "0°", "10°E"))+
    scale_y_continuous(breaks = c(45, 50, 55, 60, 65), labels = c("45°N", "50°N", "55°N", "60°N", "65°N"))
}

# gg1
# reduce the size of the plot
# options(repr.plot.width = 10, repr.plot.height = 3)
# gg1
cowplot::plot_grid(gg1, gg2, gg3, ncol = 1)

}
}

In [ ]:
if not concise:
    fixed_scale = "False"
    if fixed_scale == "False":
        md(f"**Figure {i_figure}**: Monthly mean {layer} {vv_name} for the model, observation and the difference between model and observations. For clarity, the maximum values are capped to the 98th percentiles") 
    else:
        md(f"**Figure {i_figure}**: Monthly mean {layer} {vv_name} for the model, observation and the difference between model and observations. For clarity, the maximum and minimum values are capped to cover the 98th and 2nd percentiles of both model and observations.")
    i_figure += 1

In [ ]:
if concise:
    regional = False
if regional:
    md(f"## Regional assessment of model performance for {layer_long} {vv_name}")

In [ ]:
if regional:
    md("We assessed the regional performance of the model by comparing the model with observations in a number of regions. The regions considered are mapped below.")

In [ ]:
if regional:
    lon_name = [x for x in ds_regions.to_xarray().coords if "lon" in x][0]
    lat_name = [x for x in ds_regions.to_xarray().coords if "lat" in x][0]
    df_mapped = (
        ds_regions
        .to_dataframe()
        .reset_index()
        # rename the columns
        .rename(columns = {lon_name: "lon", lat_name: "lat"})
        .melt(id_vars = ["lon", "lat"])
        .dropna()
        .merge(regions_contents.loc[:,["variable", "long_name"]])
        .drop(columns = [ "value"])
    )
    bad = ["Rosa", "Locate Shelf"]
    df_mapped = df_mapped.query("long_name not in @bad")
    xlim = np.array([df_mapped.lon.min(), df_mapped.lon.max()])
    ylim = np.array([df_mapped.lat.min(), df_mapped.lat.max()])

    def fix_name(x):
        x = x.replace("North East", "NE")
        x = x.replace("North ", "N ")
        if x == "Channel":
            x = "English Channel"
        return x

    fix_name = np.vectorize(fix_name)


    df_mapped.long_name = fix_name(df_mapped.long_name)

else:
    df_mapped = 1




In [ ]:
if regional:
    df_all = []
    df_summary = []
    for vv in ds_regions.variables:
        ds_rr = ds_regions.copy()
        ds_rr.subset(variable = vv)
        ds_rr.run()
        ds_vv = ds_ts.copy()
        time_name = [x for x in list(ds_vv.to_xarray().coords) if "time" in x][0]
        ds_vv * ds_rr
        ds_region = ds_vv.copy()
        ds_cor = ds_vv.copy()
        ds_cor.tmean()
        ds_cor.cor_space("model", "observation")
        region = list(regions_contents.query("variable == @vv").long_name)[0]
        df1 = (
            ds_cor
            .to_dataframe()
            .reset_index()
            .loc[:,["cor"]]
            .rename(columns = {"cor": "Spatial correlation"})
            .assign(region = region)
        )
        # now do the temporal correlation

        ds_cor = ds_vv.copy()
        ds_cor.cor_time("model", "observation")
        ds_cor.spatial_mean()
        df2 = (
            ds_cor
            .to_dataframe()
            .reset_index()
            .loc[:,["cor"]]
            .rename(columns = {"cor": "Temporal correlation"})
            .assign(region = region)
        )
        df = df1.merge(df2)

        ds_vv.spatial_mean()
        df_bias = (
            ds_vv
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .assign(month = lambda x: x.time.dt.month)
            .assign(bias = lambda x: x.model - x.observation)
            .assign(region = region)
            .groupby("region")
            .mean()
            .reset_index()
            .loc[:,["region", "bias"]]
        )

        # now add the RMSD, calculated in the same way as the bias
        df_rmsd = (
            ds_region
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .assign(month = lambda x: x.time.dt.month)
            .assign(rmsd = lambda x: (x.model - x.observation)**2)
            .assign(region = region)
            .groupby("region")
            .mean()
            .reset_index()
            .loc[:,["region", "rmsd"]]
            .assign(rmsd = lambda x: np.sqrt(x.rmsd))
        )

        df = df1.merge(df2).merge(df_bias).merge(df_rmsd)
        df_summary.append(df)

        


        df_vv = (
            ds_vv
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .melt("time")
            .assign(month = lambda x: x.time.dt.month)
            .assign(region = vv)
        )
        df_all.append(df_vv)
        ds_region.tmean()
        df_region = (
            ds_region
            .to_dataframe()
            .dropna()
            .reset_index()
            .loc[:,["model", "observation"]]
            .drop_duplicates()
        )
    
        del ds_rr, ds_vv, ds_region
    df_all = pd.concat(df_all).dropna()
        
    df_all = (
        df_all
        .merge(df_mapped.loc[:,["long_name", "variable"]].drop_duplicates().rename(columns = {"variable": "region"}))
    )
    df_summary= pd.concat(df_summary)

In [ ]:
# restrict df_mapped to regions in df_all
if regional:
    regions = set(df_all.query("value > 0").region)
    df_mapped = df_mapped.query("variable in @regions")

In [ ]:
%%capture --no-display
%%R -i regional -i df_mapped -i xlim -i ylim -i global_grid -w 800 -h 600
options(warn=-1)

if (regional){

    library(tidyverse)

    unique_long_names <- unique(df_mapped$long_name)
    if("Full Domain" %in% unique_long_names){
        unique_long_names <- unique_long_names[unique_long_names != "Full Domain"]
        # add Full Domain, but put it first
        unique_long_names <- c("Full Domain", unique_long_names)
        df_mapped <- df_mapped %>%
            mutate(long_name = factor(long_name, levels = unique_long_names)) 
    }

    world_map <- map_data("world")

    gg <-  ggplot(df_mapped)+
        geom_tile(aes(x = lon, y = lat))+
        coord_cartesian(xlim = xlim, ylim = ylim, expand = FALSE)+
        theme_bw(base_size = 12)+
        facet_wrap(~long_name, ncol = 5)+
        theme(axis.title.x = element_blank(),
              axis.title.y = element_blank())


gg <- gg +
    # remove the x and y axis totally
    theme(axis.text.x = element_blank(), axis.text.y = element_blank(),
          axis.ticks.x = element_blank(), axis.ticks.y = element_blank(),
          axis.title.x = element_blank(), axis.title.y = element_blank()) +
    labs(x = "", y = "") 

gg <- gg + 
        geom_polygon(data = world_map, aes(x = long, y = lat, group = group), fill = "grey", color = "grey")



    gg

}

In [ ]:
if regional:
    md(f"**Figure {i_figure}**: Regions used for validation of {layer_long} {vv_name}.")
i_figure += 1

In [ ]:
if regional:
    md(f"Time series were constructed comparing the monthly mean of the spatial average {layer_long} {vv_name} in each region. The spatial average was calculated using the mean of all grid cells within each region, accounting for grid cell area.")

In [ ]:
if regional:
    out_ts = f"../../results/regionals/{source}_{variable}_regionals.csv"
    # check if directory exists for out_ts
    if not os.path.exists("../../results/regionals"):
        os.makedirs("../../results/regionals")
    df_all.to_csv(out_ts, index = False)
if not regional:
    df_all = False

if regional:
    try:
        units = model_unit 
    except:
        units = ds_ts.contents.unit[0]
        units = str(units)

else:
    try:
        units = model_unit
    except:
        units = False

In [ ]:
%%capture --no-display
%%R -i df_all -i regional -i vv_name -i units  -w 800 -h 600
options(warn=-1)
variable = vv_name

# do a seasonal plot
if(regional){

    # convert long_name to factor
    # unique_long_names 
    unique_long_names <- unique(df_all$long_name)
    if("Full Domain" %in% unique_long_names){
        unique_long_names <- unique_long_names[unique_long_names != "Full Domain"]
        # add Full Domain, but put it first
        unique_long_names <- c("Full Domain", unique_long_names)
    df_all <- df_all %>%
        mutate(long_name = factor(long_name, levels = unique_long_names)) 
    }

    x_lab = str_glue("Spatial average {variable} ({units})")
    x_lab <- str_replace(x_lab, "/m\\^3", "m<sup>-3</sup>")
    x_lab <- str_replace(x_lab, "/m\\^2", "m<sup>-2</sup>")
    # handl /yr
    x_lab <- str_replace(x_lab, "/yr", " yr<sup>-1</sup>")
    # handle /m2
    x_lab <- str_replace(x_lab, "/m2", "m<sup>-2</sup>")
    # handle /m3
    x_lab <- str_replace(x_lab, "/m3", "m<sup>-3</sup>")
    # fix /kg to kg^-1
    x_lab <- str_replace(x_lab, "/kg", "kg<sup>-1</sup>")
    # CO2
    x_lab <- str_replace(x_lab, "CO2", "CO<sub>2</sub>")
    # O_2
    x_lab <- str_replace(x_lab, "O2", "O<sub>2</sub>")
    x_lab <- str_replace(x_lab, "O_2", "O<sub>2</sub>")
    

    library(tidyverse, warn.conflicts = FALSE)
    library(ggplot2, warn.conflicts = FALSE)
    library(ggridges, warn.conflicts = FALSE)
    library(ggthemes, warn.conflicts = FALSE)
    # make variable title
    df_all = df_all %>%
        mutate(variable = str_to_title(variable))

    gg <- ggplot(df_all)+
        geom_line(aes(x = month, y = value, colour = variable))+
        facet_wrap(~long_name, ncol = 5)+
        labs(y = x_lab)+
        labs(x = "Month")+
        theme(legend_position = "top")+
        scale_color_manual(values = c("red", "blue"))+
        scale_x_continuous(breaks = c(1, 4, 7, 10), labels = c("Jan", "Apr", "Jul", "Oct"))+ 
        theme_bw(base_size = 12)+
        labs(colour = "")+
        theme(legend.position = "top")+
        theme(axis.title.y = ggtext::element_markdown())+
        scale_color_fivethirtyeight()

        gg


}



In [ ]:
if regional:
    md(f"**Figure {i_figure}**: Seasonal cycle of {layer_long} {vv_name} for model and observations for each region. The spatial average is taken over the region.") 
    i_figure += 1

In [ ]:
if regional:
    md(f"The table below shows the average bias of sea surface {vv_name} in each region. The bias is calculated as the modelled value minus the observed value. A positive bias indicates that the model overestimates the observed value, while a negative bias indicates that the model underestimates the observed value.")

if regional:
    # output and hide index
    # first average things and tidy
    df_summary = (
        df_summary
        # .loc[:,["region", "bias"]]
        .groupby("region")
        .mean()
        .reset_index()
        # capitalize the columns
        .rename(columns = {"region": "Region", "bias": "Bias", "rmsd": "RMSD"})
    )
    # remove na values
    df_summary = df_summary.dropna().reset_index(drop = True)
    if not global_grid:
        # drop the spatial correlation
        df_summary = df_summary.drop(columns = ["Spatial correlation"])
    df_display(df_summary)

In [ ]:
if regional:
    regional_summary = [f"**Table {i_table}**: Summary of performance of the model sea surface {vv_name} in each region."]
    regional_summary.append(f"The bias ({model_unit}) column represents the spatial average of the annual mean modelled value minus the observed value.")
    regional_summary.append("The temporal correlation column represents the spatial mean of the temporal correlation between the model and observations per grid cell.") 
    if global_grid:
        regional_summary.append("The spatial correlation column represents the spatial correlation between the model and observations.")
    regional_summary = " ".join(regional_summary).replace("  ", " ")
    md(regional_summary)
    i_table = i_table + 1

In [ ]:

ds_annual = ds_model.copy()
ds_annual.rename({ds_annual.variables[0]: "model"})
ds_annual.append(ds_obs)
ds_annual.tmean()
ds_annual.merge("variables")
ds_annual.rename({ds_obs.variables[0]: "observation"})
out_dir = "../../results/annual_mean/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
out_file = out_dir + f"annualmean_{variable}_{source}.nc"
ds_annual.to_nc(out_file, zip = True, overwrite = True)
# Calculate the monthly mean and output it

ds_monthly = ds_model.copy()
ds_monthly.rename({ds_monthly.variables[0]: "model"})
ds_monthly.append(ds_obs)
ds_monthly.tmean("month")
ds_monthly.merge("variables", ["month"])
ds_monthly.rename({ds_obs.variables[0]: "observation"})
out_dir = "../../results/monthly_mean/"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)
out_file = out_dir + f"monthlymean_{variable}_{source}.nc"
ds_monthly.to_nc(out_file, zip = True, overwrite = True)

 

In [ ]:
md(f"## Can the model reproduce spatial patterns of {layer_long} {vv_name}?")

md(f"The ability of the model to reproduce spatial patterns of {layer_long} {vv_name} was assessed by comparing the modelled and observed {vv_name} at each grid cell. We calculated the Pearson correlation coefficient between the modelled and observed {vv_name} at each grid cell.")
md("This was carried out monthly and using the annual mean in each grid cell")

In [ ]:
ff = glob.glob(f"../../matched/gridded/**/**_{variable}*surface.nc")
ff = [x for x in ff if f"{source}_" in x]
if len(ff) != 1:
    raise ValueError("Something is wrong with the file")
layer = os.path.basename(ff[0]).split("_")[-1].replace(".nc", "")
ds_cor = nc.open_data(ff)
ds_cor.subset(lon = lon_lim, lat = lat_lim)
ds_cor.set_precision("F32")
ds_cor.tmean("month")
ds_cor.cor_space("model", "observation")
ds_cor_df = ds_cor.to_dataframe().reset_index()
ds_cor_df = ds_cor_df.dropna()
time_name = [x for x in list(ds_cor.to_xarray().coords) if "time" in x][0]
# rename time in dataframe
ds_cor_df.rename(columns = {time_name: "time"}, inplace = True)
# extract the month
ds_cor_df["month"] = ds_cor_df.time.dt.month
ds_cor_df = ds_cor_df.loc[:,["month", "cor"]].drop_duplicates()
# change month number to month name
ds_cor_df["month"] = ds_cor_df["month"].apply(lambda x: calendar.month_abbr[x])
# now do this annually
ds_cor = nc.open_data(ff)
ds_cor.subset(lon = lon_lim, lat = lat_lim)
ds_cor.set_precision("F32")
ds_cor.tmean("month")
ds_cor.tmean()
df_annual = ds_cor.to_dataframe()
ds_cor.cor_space("model", "observation")
ds_cor_df_annual = ds_cor.to_dataframe().reset_index()
ds_cor_df_annual = ds_cor_df_annual.dropna()
time_name = [x for x in list(ds_cor.to_xarray().coords) if "time" in x][0]
# rename time in dataframe
ds_cor_df_annual.rename(columns = {time_name: "time"}, inplace = True)
# extract the month
ds_cor_df_annual["month"] = ds_cor_df_annual.time.dt.month
ds_cor_df_annual = ds_cor_df_annual.loc[:,["month", "cor"]].drop_duplicates()
# output to csv
ds_cor_df_annual = ds_cor_df_annual.assign(month = "Annual mean")
# merge the two dataframes
ds_cor_df = pd.concat([ds_cor_df_annual, ds_cor_df])
# change month to period
ds_cor_df.rename(columns = {"month": "period"}, inplace = True)
# Give the columns more sensible names
ds_cor_df.rename(columns = {"cor": "Correlation coefficient"}, inplace = True)
ds_cor_df.rename(columns = {"period": "Time period"}, inplace = True)
# drop any na
ds_cor_df.dropna(inplace = True)
df_display(ds_cor_df)

In [ ]:
md(f"**Table {i_table}**: Pearson correlation coefficient between modelled and observed {layer_long} {vv_name} at each grid cell. The correlation was calculated monthly and using the annual mean in each grid cell.")
i_table += 1

In [ ]:
time_series = False
if regional:
    ff = glob.glob(f"../../matched/gridded/**/**_{variable}*surface.nc")
    ff = [x for x in ff if f"{source}_" in x]
    ds_ts = nc.open_data(ff)
    ds_ts.subset(lon = lon_lim, lat = lat_lim)
    years = ds_ts.years
    year_range = f"{min(years)}-{max(years)}"
    if len(years) > 1:
        ds_ts.tmean("year")
        ds_ts.run()
        time_series = True

In [ ]:
if time_series:
    md(f"The ability of the model to reproduce multi-year trends in {layer_long} {vv_name} was assessed by comparing the modelled and observed time series of annual {vv_name} across each region.")
    md(f"The figure below shows the average {vv_name} in each region")

In [ ]:
if time_series:
    df_all = []
    for vv in ds_regions.variables:
        ds_rr = ds_regions.copy()
        ds_rr.subset(variable = vv)
        ds_rr.run()
        ds_vv = ds_ts.copy()
        ds_vv * ds_rr
        ds_region = ds_vv.copy()
        ds_vv.spatial_mean()
        region = list(regions_contents.query("variable == @vv").long_name)[0]
        time_name = [x for x in list(ds_vv.to_xarray().coords) if "time" in x][0]
        df_vv = (
            ds_vv
            .to_dataframe()
            .reset_index()
            .rename(columns = {time_name: "time"})
            .loc[:,["time", "model", "observation"]]
            .melt("time")
            .assign(year = lambda x: x.time.dt.year)
            .assign(region = vv)
        )
        df_all.append(df_vv)
        ds_region.tmean()
        df_region = (
            ds_region
            .to_dataframe()
            .dropna()
            .reset_index()
            .loc[:,["model", "observation"]]
            .drop_duplicates()
        )
    
        del ds_rr, ds_vv, ds_region
    df_all = pd.concat(df_all).dropna()
        
    df_all = (
        df_all
        .merge(df_mapped.loc[:,["long_name", "variable"]].drop_duplicates().rename(columns = {"variable": "region"}))
    )

In [ ]:
%%capture --no-display
%%R -i time_series -i variable -i df_all
options(warn=-1)
if(time_series){
library(tidyverse)

ylab = str_glue("Spatial average {variable} ({units})")
ylab <- str_replace(ylab, "/m\\^3", "m<sup>-3</sup>")
ylab <- str_replace(ylab, "/m\\^2", "m<sup>-2</sup>")
# handl /yr
ylab <- str_replace(ylab, "/yr", " yr<sup>-1</sup>")
# handle /m2
ylab <- str_replace(ylab, "/m2", "m<sup>2</sup>")
# handle /m3
ylab <- str_replace(ylab, "/m3", "m<sup>3</sup>")
# fix /kg to kg^-1
ylab <- str_replace(ylab, "/kg", "kg<sup>-1</sup>")
# CO2
ylab <- str_replace(ylab, "CO2", "CO<sub>2</sub>")
# make variable title
# pco2
ylab <- str_replace(ylab, "pCO2", "pCO<sub>2</sub>")
ylab <- str_replace(ylab, "pco2", "pCO<sub>2</sub>")
# mutam should use greek letters
ylab <- str_replace(ylab, "muatm", "µatm")

# create a suitable scale_x_continuous based on year
# get the min and max year
min_year = min(df_all$year)
max_year = max(df_all$year)
# create a sequence of years, maximum is 7
ceiling((max_year - min_year)/7)
seq_year <- seq(min_year, max_year, ceiling((max_year - min_year)/7))


gg <- df_all %>%
    ggplot(aes(x = year, y = value, colour = variable))+
    geom_line()+
    facet_wrap(~long_name)+
    labs(y = ylab)+
    labs(x = "Year")+
    theme(legend.position = "top")+
    scale_color_manual(values = c("red", "blue"))+
    theme_bw(base_size = 10)+
    labs(colour = "")+
    theme(legend.position = "top")+
    theme(axis.title.y = ggtext::element_markdown())+
    scale_color_fivethirtyeight()+
    scale_x_continuous(breaks = seq_year, labels = seq_year)

    gg


}

In [ ]:
# Now do the verticals if needed
if os.path.exists(ff_vertical):
    verticals = True
    md(f"## How well does the model reproduce vertical profiles of {vv_name}?")
    text = f"The ability of the model to reproduce vertical profiles of {vv_name} was assessed by comparing the modelled and observed vertical profiles of {vv_name}. This was carried out by calculating the zonal average vertical profile of {vv_name} for both the model and observation based on annual means."
    md(text)
else:
    verticals = False

In [ ]:
try:
    regions = ds_regions.variables
    # coerce to list
    regions = list(regions)
    regions.append("full_domain")
    regionals = True
except:
    regionals = True
    regions = ["full_domain"]

In [ ]:
if time_series:
    md(f"**Figure {i_figure}**: Changes in {layer_long} {vv_name} for model and observations for each region for the period {year_range}. The spatial average is taken over the region.") 
    i_figure += 1

In [ ]:
if verticals:
    ds_verticals = nc.open_data(ff_vertical, checks = False)
    ds_verticals.tmean()
    ds_verticals.run()
    levels = ds_verticals.levels
    max_depth = max(levels)
    min_depth = min(levels)
    # 100 appropriate levels, evenly spaces
    new_levels = np.linspace(min_depth, max_depth, 100)
    ds_verticals.vertical_interp(new_levels, fixed = True)

In [ ]:
if verticals:
    df_zonals = []
    for rr in regions:
        ds_rr = ds_verticals.copy()
        if rr != "full_domain":
            ds_subset = ds_regions.copy()
            ds_subset.as_missing(0)
            ds_subset.set_fill(-9999)
            ds_subset.subset(variable = rr)
            ds_subset.run()
            long_name = ds_subset.contents.long_name[0]
            ds_subset - 1
            ds_rr + ds_subset
            ds_rr.zonal_mean()
        else:
            ds_rr.zonal_mean()
            long_name = "Full domain"
        df_merged = ds_rr.to_dataframe().reset_index(drop = True)
        depth_name = [x for x in ds_rr.to_xarray().coords if "depth" in x][0]
        # rename
        df_merged = df_merged.rename(columns = {depth_name: "depth"})
        lon_name = [x for x in ds_rr.to_xarray().coords if "lon" in x][0]
        lat_name = [x for x in ds_rr.to_xarray().coords if "lat" in x][0]
        # ditch lon
        df_merged = df_merged.drop(columns = [lon_name])
        # rename lat to lat
        df_merged = df_merged.rename(columns = {lat_name: "lat"})
        df_merged = (
            df_merged
            # melt it
            .melt(id_vars = ["depth", "lat"], value_vars = ["model", "observation"])
        )
        df_merged = (df_merged
        # variable as title
            .assign(variable = lambda x: x.variable.str.title())
        ).assign(region = long_name) 
        df_zonals.append(df_merged)
    df_zonals = pd.concat(df_zonals).dropna().reset_index(drop = True)

    # Put Full domain first
    df_zonals = pd.concat(
        [df_zonals.query("region == 'Full domain'"),
        df_zonals.query("region != 'Full domain'")]

    )

    
    # only lat, model and observation
    # df_rr = df_rr.loc[:,[lat_name, "depth", "model", "observation"]].dropna()
else:
    df_zonals = pd.DataFrame({"foo":["bar"]})

In [ ]:
%%R -i verticals -i regionals -i df_zonals -i vv_name -i units -w 800 -h 2000
# is df_zonals a dataframe

if("region" %in% colnames(df_zonals)){
    library(tidyverse)
    units <- str_replace(units, "/m\\^3", "m<sup>-3</sup>")
    units <- str_replace(units, "/m\\^2", "m<sup>-2</sup>")
    units <- str_replace(units, "/yr", " yr<sup>-1</sup>")
    units <- str_replace(units, "/m2", "m<sup>-2</sup>")
    units <- str_replace(units, "/m3", "m<sup>-3</sup>")
    units <- str_replace(units, "/kg", "kg<sup>-1</sup>")
    # CO2

    # loop through the regions
    i <- 1
    big_list = list()
    for(rr in unique(df_zonals$region)){

    gg <- df_zonals %>%
        filter(region == rr) %>%
        drop_na() %>%
        ggplot()+
        geom_raster(aes(x = lat, y = depth, fill = value))+
        facet_wrap(~variable)+
        scale_y_reverse()+
        scale_fill_viridis_c(guide  = guide_colourbar(title.position = "right"))+
        theme_bw(base_size = 10)+
        theme(legend.title = ggtext::element_markdown(angle = -90), legend.title.align = 0.5)+
        labs(x = "Latitude", y = "Depth (m)", fill = str_glue("{vv_name} ({units})"))+
        theme(axis.title.y = ggtext::element_markdown())+
        theme(axis.title.x = ggtext::element_markdown())+
        coord_cartesian(expand = FALSE)+
        # ditch the x axis labels

        labs(x = "", title = rr)

    x_breaks <- ggplot_build(gg)$layout$panel_params[[1]]$x$breaks
    x_breaks <- na.omit(x_breaks)
    x_labels <- paste0(x_breaks, "°")
    # add N to x_label is x_break > 0
    x_labels <- ifelse(x_breaks > 0, paste0(x_breaks, "°N"), ifelse(x_breaks < 0, paste0(abs(x_breaks), "°S"), "0°"))
    # add S to x_label is x_break < 0
    x_labels <- ifelse(x_breaks < 0, paste0(abs(x_breaks), "°S"), x_labels)
    # remove  the str "-" from x_labels if x_break < 0
    x_labels <- gsub("-", "", x_labels)

# no missing values
    gg1 <- gg + scale_x_continuous(breaks = x_breaks, labels= x_labels)

    # now do a bias plot
    df_bias <- df_zonals %>%
        filter(region == rr) %>%
        drop_na() %>%
        pivot_wider(names_from = variable, values_from = value) %>%
        mutate(bias = Model - Observation)
    gg2 <- df_bias %>%

        ggplot()+
        geom_raster(aes(x = lat, y = depth, fill = bias))+
        scale_y_reverse()+
        scale_fill_gradient2(low = "blue", mid = "white", high = "red", 
                             guide  = guide_colourbar(title.position = "right"))+
        theme_bw(base_size = 10)+
        theme(legend.title = ggtext::element_markdown(angle = -90), legend.title.align = 0.5)+
        labs(x = "Latitude", y = "Depth (m)", fill = str_glue("Bias ({model_unit})"))+
        theme(axis.title.y = ggtext::element_markdown())+
        theme(axis.title.x = ggtext::element_markdown())+
        coord_cartesian(expand = FALSE)
        # ditch the x axis labels
    gg2 <- gg2 + labs(x = "")
    x_breaks <- ggplot_build(gg2)$layout$panel_params[[1]]$x$breaks
    x_breaks <- na.omit(x_breaks)
    x_labels <- paste0(x_breaks, "°")
    gg2 <- gg2 + scale_x_continuous(breaks = x_breaks, labels= x_labels)

# combine the two plots, use rr as title
    gg <- cowplot::plot_grid(gg1, gg2, ncol = 1)




    big_list[[i]] <- gg
    i <- i + 1
    
    }

    cowplot::plot_grid(plotlist = big_list, ncol = 1)


}

In [ ]:
if verticals and regionals:
    md(f"**Figure {i_figure}**: Vertical profiles of {layer_long} {vv_name} for model and observations for each region. The zonal average is taken over the region.")
    i_figure += 1

In [ ]:
# delete all objects starting with ds
for name in dir():
    if name.startswith("ds"):
        del globals()[name]
for name in dir():
    if name.endswith("mask"):
        del globals()[name]
nc.cleanup()
for ff in nc.session.get_safe():
    nc.session.remove_safe(ff)
nc.cleanup()

In [ ]:
md(f"## Data Sources for validation of {vv_name}")
md_basic(definitions[variable].sources[source])

In [ ]:
if test_status:
    md("This is getting to the end!")